# Cement Plant

This dataset contains techno-economic data on a cement plant with and without carbon capture.

In [1]:
# Dependencies.
from IPython.display import HTML, Markdown

import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"
import plotly.express as px
from plotly.subplots import make_subplots

from posted import TEDF


# Set variable of TEDF.
var = "Tech|Cement Plant"

# Loading the TEDF.
tedf = TEDF.load(var)

# Define units to use.
units = {
    "Output Capacity|Cement": "t/yr",
    "CAPEX": "EUR_2024",
    "OPEX Fixed": "EUR_2024/yr",
    "Output|Cement": "t",
    "OPEX Variable": "EUR_2024",
    "Input|Electricity": "MWh",
    "Input|Coal": "MWh_coal_LHV",
}

## Fields

The techno-economic data is distinguished across the following additional fields.

### Carbon capture (`carbon_capture`)

In [2]:
Markdown(
    "\n".join(f"* **{code}**: {desc}" for code, desc in tedf.fields["carbon_capture"].codes.items())
)

* **No Capture**: Standard cement plant, without CC
* **Tail-end 20%**: Tail-end Calcium looping CC with fluidized bed reactors, 20% integration level
* **Tail-end 50%**: Tail-end Calcium looping CC with fluidized bed reactors, 50% integration level
* **Integrated CaL**: Integrated Calcium looping CC with entrained flow reactors

## Aggregated parameters

All data added to the POSTED database is aggregated automatically using the POSTED framework. The result yields the following parameters:

In [3]:
aggregated = tedf.aggregate(units=units, append_references=True)

display(
    aggregated
    .pivot(
        index=aggregated.columns[:-3],
        columns=["variable", "unit"],
        values="value",
    )
    .map(lambda x: float(f"{x:.3g}") if not pd.isnull(x) else x)
    .fillna("")
    .loc[list(tedf.fields["carbon_capture"].codes)]
)

/home/runner/work/posted/posted/posted/noslag/tedf.py:269: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



Loading ITables v2.7.0 from the internet... (need help?)


## Comparison

The figure below compares key parameters of the different technology options.

In [4]:
aggregated = tedf.aggregate(units=units)

show_variables = {"CAPEX": ["CAPEX"], "Energy Demand": ["Input|Coal", "Input|Electricity"], "Emissions": ["GHG Emissions|CO2"]}

fig = make_subplots(
    rows=1,
    cols=len(show_variables),
    subplot_titles=list(show_variables),
)

for col, (title, variables) in enumerate(show_variables.items()):
    df_rows = (
        aggregated
        .query(f"variable.isin({variables})")
    )
    
    if title == "Energy Demand":
        unit = "MWh_LHV"
    else:
        unit = df_rows["unit"].iloc[0]
    ref_unit = df_rows["reference_unit"].iloc[0]

    color_advance = sum(1 for i, v in enumerate(show_variables.values()) for _ in v if i > col)
    
    fig.add_traces(
        df_rows
        .plot.bar(
            x="carbon_capture",
            y="value",
            color="variable",
            color_discrete_sequence=px.colors.qualitative.D3[color_advance:],
        )
        .data,
        rows=1,
        cols=col+1,
    )
    
    fig.update_layout(**{
        f"xaxis{col+1 if col else ''}": dict(categoryorder="array", categoryarray=list(tedf.fields["carbon_capture"].codes)),
        f"yaxis{col+1 if col else ''}_title": f"{title}  ( {unit}/{ref_unit} )",
    })

fig.update_layout(
    legend_title=None,
    xaxis_title=None,
    barmode="stack",
)

display(fig)

/home/runner/work/posted/posted/posted/noslag/tedf.py:269: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



## Raw data

In [5]:
Markdown(f"""
The table below contains the raw data contained in the public POSTED database. This data has not be automatically normalised or harmonised in any way. You can also find this data in the GitHub repo in this file:
{link_public_github(var)}
""")


The table below contains the raw data contained in the public POSTED database. This data has not be automatically normalised or harmonised in any way. You can also find this data in the GitHub repo in this file:
<a href="https://github.com/PhilippVerpoort/posted/blob/main/posted/database/tedfs/Tech/Cement Plant.csv">posted/database/tedfs/Tech/Cement Plant.csv</a>


In [6]:
tedf.edit_data()

Loading ITables v2.7.0 from the internet... (need help?)
